In [36]:
import torch
from IPython import display
from d2l import torch as d2l

In [37]:
batch_size = 256
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size)

In [38]:
# 初始化模型参数
num_inputs = 784  # 28*28
num_outputs = 10

W = torch.normal(0, 0.01, size=(num_inputs, num_outputs), requires_grad=True)
b = torch.zeros(num_outputs, requires_grad=True)

In [39]:
def softmax(X):
    X_exp = torch.exp(X)
    partition = X_exp.sum(1, keepdim=True)  # 假设X是（50，10）保留列的维度，结果仍然是二维
    return X_exp / partition

In [40]:
X = torch.normal(0, 1, (2, 5))
X_prob = softmax(X)
X, X_prob, X_prob.sum(axis=1, keepdim=True) 

(tensor([[-2.1728,  0.0570, -0.8389,  0.3236, -1.0046],
         [-1.6183, -0.0281,  1.2581,  0.8333, -1.2075]]),
 tensor([[0.0340, 0.3157, 0.1289, 0.4122, 0.1092],
         [0.0272, 0.1334, 0.4827, 0.3157, 0.0410]]),
 tensor([[1.],
         [1.]]))

In [41]:
# 定义模型
def net(X):
    return softmax(torch.matmul(X.reshape(-1, W.shape[0]), W) + b)

In [42]:
# 高级索引
y = torch.tensor([0, 2])
y_hat = torch.tensor([[0.1, 0.3, 0.6], [0.3, 0.2, 0.5]])
y_hat[[0, 1], y]  # 选择y_hat[0][0] and y_hat[1][2]

tensor([0.1000, 0.5000])

In [43]:
# y_hat = y_hat.argmax(axis=1)
# y_hat  # 预测结果，一维

In [44]:
y_hat.shape, len(y_hat.shape)  # len返回第一维的长度

(torch.Size([2, 3]), 2)

In [45]:
# 定义损失函数
def cross_entropy(y_hat, y):
    return -torch.log(y_hat[range(len(y_hat)), y])

cross_entropy(y_hat, y)

tensor([2.3026, 0.6931])

In [46]:
# 分类精度
def accuracy(y_hat, y):
    """计算预测正确的数量"""
    if len(y_hat.shape) > 1 and y_hat.shape[1] > 1:  # 如果是矩阵（多分类任务）
        y_hat = y_hat.argmax(axis=1)  # 一维数据
    cmp = y_hat.type(y.dtype) == y
    return float(cmp.type(y.dtype).sum())

In [47]:
accuracy(y_hat, y) / len(y)

0.5

In [48]:
def evaluate_accuracy(net, data_iter):
    """计算在指定数据集上模型的精度"""
    if isinstance(net, torch.nn.Module):
        net.eval()  # 设置为评估模式，dropout/bartchnorm等的策略有所不同
    metric = Accumulator(2)  # [正确预测数，预测总数]
    with torch.no_grad():
        for X, y in data_iter:
            metric.add(accuracy(net(X), y), y.numel())
    return metric[0] / metric[1]


In [49]:
class Accumulator:
    """在 n 个变量上累加"""
    def __init__(self, n):
        self.data = [0.0] * n  # 长度为 n 的列表
    
    def add(self, *args):
        self.data = [a + float(b) for a, b in zip(self.data, args)]

    def reset(self):
        self.data = [0.0] * len(seld.data)

    def __getitem__(self, idx):
        return self.data[idx]

In [51]:
evaluate_accuracy(net, test_iter)

0.053

In [52]:
# 训练
def train_epoch_ch3(net, train_iter, loss, updater):
    """训练模型一个迭代周期"""
    # 将模型设置为训练模式
    if isinstance(net, torch.nn.Module):
        net.train()
    # 训练损失总和、训练准确度总和、样本数
    metric = Accumulator(3)
    for X, y in train_iter:
        # 计算损失
        y_hat = net(X)
        l = loss(y_hat, y)
        if isinstance(updater, torch.optim.Optimizer):
            # 使用PyTorch内置的优化器和损失函数
            updater.zero_grad()
            l.mean().backward()
            updater.step()
        else:
            # 使用定制的优化器和损失函数
            l.sum().backward()
            updater(X.shape[0])  # ?
        metric.add(float(l.sum()), accuracy(y_hat, y), y.numel())
    return metric[0] / metric[2], metric[1] / metric[2]  # 返回训练损失和训练精度

In [ ]:
class Animator:
    """在动画中绘制数据"""
    def __init__(self, xlabel=None, ylabel=None, legend=None, xlim=None,
                 ylim=None, xscale='linear', yscale='linear',
                 fmts=('-', 'm--', 'g-.', 'r:'), nrows=1, ncols=1,
                 figsize=(3.5, 2.5)):
        # 增量地绘制多条线
        if legend is None:
            legend = []
        d2l.use_svg_display()
        self.fig, self.axes = d2l.plt.subplots(nrows, ncols, figsize=figsize)
        if nrows * ncols == 1:
            self.axes = [self.axes, ]
        # 使用lambda函数捕获参数
        # 使用lambda的原因：在需要时灵活调用？不是很理解
        self.config_axes = lambda:d2l.set_axes(
            self.axes[0], xlabel, ylabel, xlim, ylim, xscale, yscale, legend
        )
        self.X, self.Y, self.fmts = None, None, fmts
    
    def add(self, x, y):
        # 向图标中添加多个数据点
        if not hasattr(y, "__len__"):
            y = [y]
        n = len(y)
        if not hasattr(x, "__len__"):
            x = [x] * n
        if not self.X:  # 如果X为空
            self.X = [[] for _ in range(n)]  # _用作一个临时变量名，表示该变量在后续代码中不会被使用
        if not self.Y:
            self.Y = [[] for _ in range(n)]
        if i, (a, b) in enumerate(zip(x, y)):
            if a is not None and b is not None:
                self.X[i].append(a)
                self.Y[i].append(b)
        self.axes[0].cla()
        for x, y, fmt in zip(